#**Maestría en Inteligencia Artificial Aplicada**
##**Advanced Machine Learning Methods (Gpo 10)**
###Tecnológico de Monterrey
###Profesor Ph.D. José Antonio Cantoral Ceballos

## **Activity 4**

### Text Generator

##**Team 33**:

### Humberto Lozano Cedillo A01363184
### Julio Cesar Lynn Jimenez A01793660
### Sarah Mendoza Medina A01215352
### David Mireles Samaniego A01302935

## TC 5033
### Text Generation

<br>

#### Activity 4: Building a Simple LSTM Text Generator using WikiText-2
<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation. 

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function. 

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



In [4]:
import numpy as np
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import random
import time

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
train_dataset, val_dataset, test_dataset = WikiText2()

In [7]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

In [8]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [9]:
seq_length = 50
def data_process(raw_text_iter, seq_length = 50):
    data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]
    data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
#     target_data = torch.cat(d)
    return (data[:-(data.size(0)%seq_length)].view(-1, seq_length), 
            data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))  

# # Create tensors for the training set
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

In [10]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

In [11]:
batch_size = 64  # choose a batch size that fits your computation resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [12]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        return decoded, hidden

    def init_hidden(self, batch_size):

        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))



vocab_size = len(vocab) # vocabulary size
emb_size = 300 # embedding size
neurons = 128 # the dimension of the feedforward network model, i.e. # of neurons 
num_layers = 2 # the number of nn.LSTM layers
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


In [25]:
def accuracy(model, loader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for i, (data, targets) in enumerate(loader):
            # Detach the hidden state between batches
            hidden = model.init_hidden(batch_size) 
            
            xi = data.to(device=device, dtype = torch.int32)
            yi = targets.to(device=device, dtype = torch.long) 
            yi_flatten = yi.view(-1)

            y_pred, hidden = model(xi, hidden)
            y_pred_max_prob, y_pred_max_idx = y_pred.max(dim=2)

            total_acc += (y_pred_max_idx.view(-1) == yi_flatten).sum().item()
            total_count += yi_flatten.size(0)
            
    return total_acc / total_count

In [24]:
def train(model, epochs, optimiser, loss_function):
    '''
    The following are possible instructions you may want to conside for this function.
    This is only a guide and you may change add or remove whatever you consider appropriate
    as long as you train your model correctly.
        - loop through specified epochs
        - loop through dataloader
        - don't forget to zero grad!
        - place data (both input and target) in device
        - init hidden states e.g. hidden = model.init_hidden(batch_size)
        - run the model
        - compute the cost or loss
        - backpropagation
        - Update paratemers
        - Include print all the information you consider helpful
    
    '''
    
    model = model.to(device=device)
    
    for epoch in range(epochs):

        total_acc, total_count = 0, 0
        log_interval = int(len(train_loader) / 5)
        start_time = time.time()
        model.train()

        for i, (data, targets) in enumerate(train_loader):
            # Detach the hidden state between batches
            hidden = model.init_hidden(batch_size) 
            
            # data_words = []
            # for i in range(data.size(0)):
            #     for j in range(data.size(1)):
            #         idx = data[i][j]
            #         data_words.append(vocab.lookup_token(idx))
            #     data_words.append('|'*4)
            # data_str = ' '.join(data_words)
            # print(data_str)
            

            # target_words = []
            # for i in range(targets.size(0)):
            #     for j in range(targets.size(1)):
            #         idx = targets[i][j]
            #         target_words.append(vocab.lookup_token(idx))
            #     target_words.append('|'*4)
            # target_str = ' '.join(target_words)
            # print('-'*40)
            # print(target_str)
        
            xi = data.to(device=device, dtype = torch.int32)
            yi = targets.to(device=device, dtype = torch.long) # Needs to be long for the loss_function treat the target as class indices
            yi_flatten = yi.view(-1)
            optimiser.zero_grad()

            y_pred, hidden = model(xi, hidden)
            y_pred_max_prob, y_pred_max_idx = y_pred.max(dim=2)

            # input needs to match the yi flattened size in one dimension and the vocab_size (our classes) on another
            cost = loss_function(input=y_pred.view(batch_size*seq_length, vocab_size), target=yi_flatten)
            cost.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
            optimiser.step()

            total_acc += (y_pred_max_idx.view(-1) == yi_flatten).sum().item()
            total_count += yi_flatten.size(0)
            
            if i % log_interval == 0 and i > 0:
                print(
                    "| epoch {:3d} | {:5d}/{:5d} batches "
                    "| accuracy {:8.3f}".format(
                        epoch, i, len(train_loader), total_acc / total_count
                    )
                )
            total_acc, total_count = 0, 0

        accu_val = accuracy(model, val_loader)

        print("-" * 59)
        print(
            "| end of epoch {:3d} | time: {:5.2f}s | "
            "valid accuracy {:8.3f} ".format(
                epoch, time.time() - start_time, accu_val
            )
        )
        print("-" * 59)
                    

In [26]:
# Call the train function
loss_function = nn.CrossEntropyLoss()
lr = 0.0005
epochs = 5
optimiser = optim.Adam(model.parameters(), lr=lr)
train(model, epochs, optimiser, loss_function)

| epoch   0 |   128/  640 batches | accuracy    0.151
| epoch   0 |   256/  640 batches | accuracy    0.142
| epoch   0 |   384/  640 batches | accuracy    0.147
| epoch   0 |   512/  640 batches | accuracy    0.165
-----------------------------------------------------------
| end of epoch   0 | time: 49.72s | valid accuracy    0.173 
-----------------------------------------------------------
| epoch   1 |   128/  640 batches | accuracy    0.165
| epoch   1 |   256/  640 batches | accuracy    0.172
| epoch   1 |   384/  640 batches | accuracy    0.168
| epoch   1 |   512/  640 batches | accuracy    0.180
-----------------------------------------------------------
| end of epoch   1 | time: 49.85s | valid accuracy    0.187 
-----------------------------------------------------------
| epoch   2 |   128/  640 batches | accuracy    0.181
| epoch   2 |   256/  640 batches | accuracy    0.188
| epoch   2 |   384/  640 batches | accuracy    0.189
| epoch   2 |   512/  640 batches | accuracy

In [29]:
def generate_text(model, start_text, num_words, temperature=1.0):
    '''
    model.eval()
    words = tokeniser(start_text)
    hidden = model.init_hidden(1)
    for i in range(0, num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden)
        last_word_logits = y_pred[0][-1]
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(vocab.lookup_token(word_index))

    return ' '.join(words)
    '''
    
    model.eval()
    words = tokeniser(start_text)
    hidden = model.init_hidden(1)
    for i in range(0, num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden)
        last_word_logits = y_pred[0][-1]
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(vocab.lookup_token(word_index))

    return ' '.join(words)

# Generate some text
print(generate_text(model, start_text="I like", num_words=100, temperature=0.9))


i like the manufacture was renamed the image of train . in the time , 1989 , <unk> in the 20th century , he other smaller runs , or later the past . = = = = = = = = = = = = the earthquake of 122 noisy then role at the spider jacob times with the less professional age . the parkway for the rain with at the eastern night , as federer went to the english mountains of his performance , while the word is a <unk> and send that ships and the ambiguous trust in the <unk>


In [32]:
print(generate_text(model, start_text="Mexico (Spanish: México), officially the United Mexican States, is a  ", num_words=100, temperature=0.9))

mexico ( spanish méxico ) , officially the united mexican states , is a of people for many <unk> and shifted gold for the <unk> of the <unk> , he who was money . building canaan oscar to the single , the churchyard saw the intersection as a track and the tuition , and it to get the right into support and inventory . , they ' s are captain , such as future <unk> , was expressed operating in the ninth river in the port , and the historic @-@ u education and that it said that they were a real what wished to love . it was played one of the episode
